In [1]:
import sqlite3
import telebot
from telebot import types

# Активация бота + подключение к БД
bot = telebot.TeleBot("telebot_token")

@bot.message_handler(commands=['start'])
def start_message(message):
  user_id = message.from_user.id
  cursor.execute("SELECT * FROM hads_answers where user_id=?",(user_id,))
  records = cursor.fetchone()
  if records == None:
      cur_question = 1
      cursor.execute('INSERT INTO hads_answers (user_id,cur_question) VALUES (?,?)', (user_id,cur_question,))
      conn.commit()
      bot.send_message(message.chat.id, 'Добро пожаловать.\nДанный бот предназначен для прохождения теста HADS - Госпитальная Шкала Тревоги и Депрессии.\nДля начала тестирования воспользуйтесь командой /hads\nТак же вы можете ознакомиться с рекомендациями для прохождения теста по команде /rules.\nЕсли вы забудите команды бота, то воспользуйтесь командой /help.')
  else:
      bot.send_message(message.chat.id, 'Добро пожаловать.\nДанный бот предназначен для прохождения теста HADS - Госпитальная Шкала Тревоги и Депрессии.\nДля начала тестирования воспользуйтесь командой /hads\nТак же вы можете ознакомиться с рекомендациями для прохождения теста по команде /rules.\nЕсли вы забудите команды бота, то воспользуйтесь командой /help.')
conn = sqlite3.connect('telebot.db', check_same_thread=False)
cursor = conn.cursor()

@bot.message_handler(commands=['rules'])
def rules(message):
  bot.send_message(message.chat.id, 'Практические рекомендации для прохождения теста.\nШкалу необходимо заполнить за 10–15 минут самостоятельно, без обсуждения с кем-либо. Каждому утверждению шкалы HADS соответствуют четыре варианта ответа. Выберите и отметьте тот из ответов, который соответствует Вашему состоянию в течении последних 7 дней. Не раздумывайте долго над ответом на вопрос. Ваша первая реакция будет наиболее верной.\n Для начала теста воспользуйтесь командой - /hads.')

@bot.message_handler(commands=['help'])
def help(message):
  bot.send_message(message.chat.id, 'Команды для бота:\n/start - Команда необходимая для запуска бота.\n/hads - Команда необходимая для запуска тестирования.\n/reset - Команда для сброса результатов тестирования.\n/result - Команда для отображения результатов тестирования.\n/rules - Команда для отображения рекомендаций по прохождению тестирования.\n/help - Команда необходимая для отображения все команд бота.')

@bot.message_handler(commands=['reset'])
def reset_score(message):
  user_id = message.from_user.id
  cur_question = 1
  p1_sum = 0
  p2_sum = 0
  sum = 0
  cursor.execute('UPDATE hads_answers set cur_question=?,p1_sum=?,p2_sum=?,sum=? where user_id=?',(cur_question,p1_sum,p2_sum,sum,user_id,))
  bot.send_message(message.chat.id, 'Ваши баллы сброшены!\nТеперь вы можете еще раз пройти тест.\nДля этого напишите /hads')
  conn.commit()

@bot.message_handler(commands=['result'])
def reset_score(message):
  user_id = message.from_user.id
  cursor.execute('SELECT p1_sum,p2_sum,sum FROM hads_answers where user_id=?',(user_id,))
  results = cursor.fetchone()
  sum_points = results[0]
  sum_points_2 = results[1]
  sum = results[2]
  if sum_points >= 8 or sum_points_2 >= 8:
    if sum >= 8 and sum <= 10:
      bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует субклинически выраженная тревога / депрессия.\nТак же вы набрали по одной из шкал в сумме более 8 баллом, что означает что вам необходима консультация врача.', reply_markup=types.ReplyKeyboardRemove())
    if sum >= 11:
      bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует клинически выраженная тревога / депрессия.\nТак же вы набрали по одной из шкал в сумме более 8 баллом, что означает что вам необходима консультация врача.', reply_markup=types.ReplyKeyboardRemove())
  else:
    if sum <= 7:
      bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас отсутсвуют достоверно выраженные симптомы тревоги и депрессии.', reply_markup=types.ReplyKeyboardRemove())
    if sum >= 8 and sum <= 10:
      bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует субклинически выраженная тревога / депрессия.', reply_markup=types.ReplyKeyboardRemove())
    if sum >= 11:
      bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует клинически выраженная тревога / депрессия.', reply_markup=types.ReplyKeyboardRemove())
  conn.commit()

@bot.message_handler(commands=['hads'])
def select(message):
  user_id = message.from_user.id
  cursor.execute("SELECT * FROM hads_answers where user_id=?",(user_id,))
  records = cursor.fetchone()
  if records == None:
      bot.send_message(message.chat.id, 'Просим прощения, база данных была обнулена, для начала работы с ботом напишите еще раз команду /start')
  else:
    cursor.execute("SELECT * FROM hads_answers where user_id=?",(user_id,))
    user_info = cursor.fetchone()
    cur_question = user_info[1]
    if cur_question == 15:
      bot.send_message(message.chat.id, 'Вы уже прошли тест! Для сброса результатов попытки напишите команду /reset, если хотите посмотреть результаты вашей предыдущей попытки напишите команду /result')
    else:
      cursor.execute("SELECT * FROM hads_question where id=?",(cur_question,))
      records = cursor.fetchone()
      question = records[1]
      asnwers = records[2]
      asn = asnwers.split(';') 
      keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True,one_time_keyboard=True)
      for button_content in asn:
        btn =  types.KeyboardButton(button_content)
        keyboard.add(btn)
      msg = bot.send_message(message.chat.id, text=question, reply_markup=keyboard)
      bot.register_next_step_handler(msg, on_selection)

def on_selection(message):
  selection = message.text
  user_id = message.from_user.id
  cursor.execute("SELECT * FROM hads_answers where user_id=?",(user_id,))
  user_info = cursor.fetchone()
  cur_question = user_info[1]
  sum_points = user_info[16]
  sum_points_2 = user_info[17]
  field_names = [i[0] for i in cursor.description]
  cursor.execute("SELECT * FROM hads_question where id=?",(cur_question,))
  records = cursor.fetchone()
  asnwers = records[2]
  answer_point = records[3]
  points = answer_point.split(sep=';')
  asn = asnwers.split(';') 
  if selection in asn:  
    asn_index = asn.index(selection)
    cur_question += 1
    if cur_question <= 8:
      sum_points = sum_points + int(points[asn_index])
      cursor.execute("UPDATE hads_answers set " + str(field_names[cur_question])+"=?, cur_question=?, p1_sum=? where user_id=?",(points[asn_index],cur_question,sum_points,user_id,))
      conn.commit()
      select(message)
    if cur_question > 8 and cur_question < 15:
      sum_points_2 = sum_points_2 + int(points[asn_index])
      cursor.execute("UPDATE hads_answers set " + str(field_names[cur_question])+"=?, cur_question=?, p2_sum=? where user_id=?",(points[asn_index],cur_question,sum_points_2,user_id,))
      conn.commit()
      select(message)
    if cur_question == 15:
      sum_points_2 = sum_points_2 + int(points[asn_index])
      sum = sum_points + sum_points_2
      cursor.execute("UPDATE hads_answers set " + str(field_names[cur_question])+"=?, cur_question=?, p2_sum=?,sum=? where user_id=?",(points[asn_index],cur_question,sum_points_2,sum,user_id,))
      conn.commit()
      if sum_points >= 8 or sum_points_2 >= 8:
        if sum >= 8 and sum <= 10:
          bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует субклинически выраженная тревога / депрессия.\nТак же вы набрали по одной из шкал в сумме более 8 баллов, что означает что вам необходима консультация врача.', reply_markup=types.ReplyKeyboardRemove())
        if sum >= 11:
          bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует клинически выраженная тревога / депрессия.\nТак же вы набрали по одной из шкал в сумме более 8 баллов, что означает что вам необходима консультация врача.', reply_markup=types.ReplyKeyboardRemove())
      else:
        if sum <= 7:
          bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас отсутсвуют достоверно выраженные симптомы тревоги и депрессии.', reply_markup=types.ReplyKeyboardRemove())
        if sum >= 8 and sum <= 10:
          bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует субклинически выраженная тревога / депрессия.', reply_markup=types.ReplyKeyboardRemove())
        if sum >= 11:
          bot.send_message(message.chat.id, 'Вы набрали '+str(sum)+' баллов.\nПо вашим баллам можно сказать о том, что у вас присутствует клинически выраженная тревога / депрессия.', reply_markup=types.ReplyKeyboardRemove())
  else:
    bot.send_message(message.chat.id, 'Пожалуйста, используйте кнопки для ответов!\nПерезапустите тест!\nЕсли вы забыли команды, то воспользуйтесь командой /help.',reply_markup=types.ReplyKeyboardRemove())

# Запуск бота
bot.polling(none_stop=True, interval=0)